# libraries

In [ ]:
# general libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import yaml
import requests
import time

# analysis libraries
import osmnx as ox
import momepy as mm
import random
from shapely.geometry import Point, LineString

# visualization libraries
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [ ]:
# directories
datasets_dir = "datasets/"
overpass_url = "http://overpass-api.de/api/interpreter"
req_path = datasets_dir + "osmdata_requests.yaml"
open_path = datasets_dir + "opendata_requests.yaml"

In [ ]:
# check if directory already exist, otherwise create them
if os.path.exists(datasets_dir):
    pass
else:
    datasets_dir = "datasets/"
    os.makedirs(datasets_dir, exist_ok=True)

In [ ]:
os.makedirs("images", exist_ok=True)

# extract all data

## collect polygons data

In [ ]:
raw_polygons = gpd.read_file(datasets_dir + "bologna_polygons.geojson")

In [ ]:
raw_polygons= raw_polygons.set_crs(epsg=4326, allow_override=True)
data_polygons = raw_polygons.to_crs(epsg=32632)

In [ ]:
data_polygons.plot(figsize=(10, 10), edgecolor="w").set_axis_off()

In [ ]:
# compute area
data_polygons['area'] = data_polygons['geometry'].area / 1e6 # better readability (squared kilometers)

# show df
data_polygons

## collect data related to openstreetmap
optional: to run only when raw data do not exist
NB: we retrieve data using the EPSG:32632 coordinates, which refers to WGS 84 / UTM zone 32N (uses meters).

In [ ]:
# OPENSTREETMAP DATA

# read the request file
with open(req_path, 'r') as f:
    data_requests = yaml.safe_load(f)

# initialize the empty dataframe
all_data = pd.DataFrame(columns=["id", "lat", "lon", "class"])

# for each urban function retrieve the corresponding data points from openstreetmap
for el in data_requests:
    response = requests.get(overpass_url,
                            params={'data': data_requests[el]})
    #print(response)
    raw_data = response.json()
    elements = raw_data.get('elements', [])

    rows = []
    for item in elements:
        # for some urban functions we retrieve the points directly as nodes
        if item.get('type') == 'node':
            lat = item.get('lat')
            lon = item.get('lon')
            if lat is None or lon is None:
                continue

        # for other instead we retrieve the area and then identify the coordinates of the centroid
        elif 'center' in item:
            center = item['center']
            lat = center.get('lat')
            lon = center.get('lon')
            if lat is None or lon is None:
                continue
        else:
            continue
        
        rows.append({
            'id': item.get('id'),
            'lat': lat,
            'lon': lon,
            'class': el
        })

    gdf_tmp = pd.DataFrame(rows)
    all_data = pd.concat([all_data, gdf_tmp], ignore_index=True)

    print(f"OSM data retrieved correctly for {el} urban function: {gdf_tmp.shape[0]} nodes/areas were collected")
    time.sleep(3)  # to avoid blocking from Overpass API

    print(f"current total count of PoIs: {all_data.shape[0]}")

final_gdf_osm = gpd.GeoDataFrame(
    all_data,
    geometry=gpd.points_from_xy(all_data.lon, all_data.lat),
    crs="EPSG:32632"
)

print(f"For the municipality of Bologna, {final_gdf_osm.shape[0]} Points of Interest related to OpenStreetMap were collected.")


## collect data from Bologna OpenData Portal

In [ ]:
# OPEN THE DATASET
with open(open_path, "r") as f:
    opendata_requests = yaml.safe_load(f)

In [ ]:
# RETRIEVE ALL DATA from Bologna Open Data Portal
def retrieve_open_data(base_url):
    
    # here we cannot retrieve all data together so we create multiple batches
    offset = 0
    limit = 100
    all_records = []

    while True:
        
        paged_url = f"{base_url}&limit={limit}&offset={offset}" # change everytime the batches
        
        response = requests.get(paged_url) # extract the raw data
        json_resp = response.json() 
        records = json_resp.get("results", []) # extract the data we actually want

        if not records:
            break

        all_records.extend(records)

        if len(records) < limit: # reached the limit of data available
            break

        offset += limit
    
    return pd.DataFrame(all_records)


In [ ]:
# FUNCTION TO ASSIGN A RANDOM UNIQUE ID
def assign_unique_4digit_ids(df, used_ids=set()):
    n = len(df)
    possible_ids = set(range(1000, 10000)) - used_ids

    if len(possible_ids) < n:
        raise ValueError("not enough unique 4-digit IDs left to assign")

    unique_ids = random.sample(list(possible_ids), n)

    df = df.copy()
    df['id'] = unique_ids

    return df, used_ids.union(unique_ids)



In [ ]:
used_ids = set() # the initial pool of ids is empty 

# we extract two different datasets from the Bologna Open Data Portal

# -----------------------------------------------------
# GREEN URBAN FUNCTION EXTRACTION
green_df = retrieve_open_data(opendata_requests["green"][0]) # retrieve entire dataframe
parchi_df = green_df[green_df['tipo'].isin(['PARCO', 'GIARDINI'])] # extract only public gardens and parks

parchi_df, used_ids = assign_unique_4digit_ids(parchi_df, used_ids) # assign unique ids
parchi_df['class'] = 'green'
parchi_df['lat'] = parchi_df['geo_point_2d'].apply(lambda x: x['lat'])
parchi_df['lon'] = parchi_df['geo_point_2d'].apply(lambda x: x['lon'])

parchi_df = parchi_df[['id', 'lat', 'lon', 'class']]
parchi_gdf = gpd.GeoDataFrame(parchi_df,
                             geometry=gpd.points_from_xy(parchi_df.lon, parchi_df.lat),
                             crs="EPSG:32632")


# combine all the data points retrieved

In [ ]:
combined_df = pd.concat([final_gdf_osm, parchi_gdf], ignore_index=True) # combine all the datasets
combined_df = combined_df.set_crs('EPSG:4326', allow_override=True) # change crs
combined_df # check the dataframe

In [ ]:
# PLOT BY URBAN FUNCTION
fig = px.scatter_map(combined_df,
                        lat='lat',
                        lon='lon',
                        color='class',
                        hover_name='id',
                        zoom=12,
                        map_style='open-street-map',
                        title=f'Distribution of PoIs retrieved based on urban function'
                     )
fig.show() #renderer='notebook'

# data cleaning
Here we want to check (and possibly remove) data points that could be doubled or replicated. To do so, we work by class, urban function, and try to detect where more than three points are either inside a small buffer (50 m) or aligned on the same street for more than 50 meters. 
This could imply that such data points refer to the same original structure but have multiple facilities concentrated in the same area.
The only exception we consider is schools, where it is common that inside the same building are present different school grades (elementary, middle...) -> here, even the function is the same, the type of school is different and we want to keep this type of information inside our analysis.

In [ ]:
grouped_points = []
group_id = 0

# remove education from dataset
combined_df_noedu = combined_df[combined_df['class'] != 'education'].copy()
combined_df_noedu = combined_df_noedu.to_crs(epsg=32632)

# now for each class
for cls, group in combined_df_noedu.groupby('class'):
    
    # and for each data point
    for idx, row in group.iterrows():
        
        # create a buffer of 30 meters
        buffer = row.geometry.buffer(50)
        
        # count how many elements are inside this buffer
        nearby = group[group.geometry.within(buffer)]
        
        # if more than 3 elements
        if len(nearby) >= 3:
            temp = nearby.copy()
            temp["group_id"] = group_id # assign a common identification number
            grouped_points.append(temp) # add it to our dataframe
            group_id += 1 # with a group id

# now we merge the close data points such that it becomes a single data point
flagged_points = pd.concat(grouped_points).drop_duplicates(subset=['id'])

centroids = (
    flagged_points
    .dissolve(by='group_id', as_index=False)
    .copy()
)
centroids['geometry'] = centroids.geometry.centroid

In [ ]:
# visual check of what we're doing
flagged_points = flagged_points.to_crs(epsg=4326)

# PLOT BY URBAN FUNCTION
fig = px.scatter_map(flagged_points,
                        lat='lat',
                        lon='lon',
                        color='class',
                        hover_name='id',
                        zoom=12,
                        map_style='open-street-map',
                        title=f'Distribution of too close data points (before cleaning: {flagged_points.shape[0]})'
                     )
fig.show() 

What we see:
- administration has three data points grouped very close. Two of them, along Piazza Galileo Galilei have also the same name ("Questura"), while the third one along Via Quattro Novembre is the Municipality Palace. Can be maintained separated.
- healthcare has three facilities (appeared to be pharmacies) very close referring to different commercial activites, thus they should be separaterd.

# nodes - polygons alignment
For further analysis we're going to consider the polygons separately therefore it is important to have a correspondence between nodes and polygons.

In [ ]:
# initial settings
data_proj = combined_df.to_crs(epsg=32632)
polygons_proj = data_polygons.to_crs(epsg=32632)
polygons_proj.id = polygons_proj.id.astype(str)

In [ ]:
# SPATIAL JOIN: we want to have a column in data dataset that identifies the grid polygon that contains each node
new_data = gpd.sjoin(data_proj, polygons_proj, how="left", predicate="within")
new_data = new_data.rename(columns={
    "id_right": "polygon_id",
    "id_left": "node_id"})


In [ ]:
new_data 

In [ ]:
# CHECK FOR OUTSIDE BOUNDARIES DATA
nan_data = new_data[new_data.polygon_id.isna()]
print(f"data not in any polygon: {nan_data.shape[0]}")

new_data = new_data[~new_data.polygon_id.isna()]
print(f"data kept: {new_data.shape[0]}")

In [ ]:
new_data 

In [ ]:
new_data = new_data [['node_id','lat','lon','class','geometry','polygon_id']]

# data visualization
Let's plot the data retrieved

In [ ]:
polygons_gdf = data_polygons.to_crs(epsg=4326) # change to unprojected coordinates

# scatter map for the data points retrieved
scatter_points = go.Scattermap(
    lat=new_data['lat'],
    lon=new_data['lon'],
    mode='markers',
    marker=dict(size=8, color='green'),
    text=new_data['node_id'], 
    name='POIs'
)

# here we need to convert the polygons into plotly figures (still polygons!)
polygons_traces = []

for _, row in polygons_gdf.iterrows():
    geom = row.geometry
    name = row.get('polygon_id')  

    coords = list(geom.exterior.coords)
    lons, lats = zip(*coords)

    polygons_traces.append(go.Scattermap(
        lat=lats,
        lon=lons,
        mode='lines',
        fill='toself',
        fillcolor='rgba(0, 100, 0, 0.2)',
        line=dict(color='darkgreen'),
        name=name
        ))


# create open-street-map layout
layout = go.Layout(
    mapbox=dict(
        style="open-street-map",
        zoom=12,
        center=dict(lat=combined_df['lat'].mean(), lon=combined_df['lon'].mean())
    ),
    title="PoIs distribtuion in the city of Bologna"
)

# actual plot
fig = go.Figure(data=[scatter_points] + polygons_traces, layout=layout)
fig.show()


# data saving

In [ ]:
# STORE COMPLETE DATASET

geojson_path = os.path.join(datasets_dir, "geopoints_data.geojson")
new_data.to_file(geojson_path, driver='GeoJSON')

print(f"dataset stored in directory: {geojson_path}")

# detailed information on dataset
Here we want to gather information on the original polygon data and their buffered versions.

## ORIGINAL POLYGONS PLOT

In [ ]:
# DATA POINTS DISTRIBUTION BY ORIGINAL POLYGONS
poi_count = new_data.groupby("polygon_id").count()
poi_count

In [ ]:
# for plot purposes we change the crs system Web Mercator (EPSG:3857) -> to match basemap tiles
data_polygons = data_polygons.to_crs(epsg=3857)
data_polygons.id = data_polygons.id.astype(str)
 
# plot
fig, ax = plt.subplots(figsize=(10, 10))

data_polygons.plot(
    ax=ax,
    column='id',          
    cmap='Set2',        
    alpha=0.5,            
    edgecolor='black',    
    linewidth=0.5,
    legend=True)



ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)  

ax.set_axis_off()

plt.title("Polygons identified in the municipality of Bologna", fontsize=14)
plt.tight_layout()

# to store
output_path = os.path.join("images", "bologna_polygons.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# plot
plt.show()


## 100 M BUFFER POLYGONS PLOT

In [ ]:
# PREPARATIONS
buffered_100 = []

for el in data_polygons.id.unique():
    
    # here we compute the buffered polygon
    geom_proj = polygons_proj.loc[int(el)].geometry.buffer(100) # add a 100 meters buffer to the polygon
    
    # and look for the data points contained in it
    buffered_polygon = gpd.GeoDataFrame(geometry=[geom_proj], crs="EPSG:32632")
    subset_data_buffered = gpd.sjoin(new_data, buffered_polygon, predicate="within")
    
    buffered_100.append({'id': el, 'geometry': geom_proj, 'POIs': subset_data_buffered.shape[0]})
    

In [ ]:
buffered100_polygons = gpd.GeoDataFrame(buffered_100, crs="EPSG:32632")
buffered100_polygons['area'] = buffered100_polygons.geometry.area /1e6
buffered100_polygons

In [ ]:
# for plot purposes we change the crs system Web Mercator (EPSG:3857) -> to match basemap tiles
buffered50_polygons = buffered50_polygons.to_crs(epsg=3857)
buffered50_polygons.id = buffered50_polygons.id.astype(str)
 
# plot
fig, ax = plt.subplots(figsize=(10, 10))

buffered50_polygons.plot(
    ax=ax,
    column='id',          
    cmap='Set2',        
    alpha=0.5,            
    edgecolor='black',    
    linewidth=0.5,
    legend=True)



ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)  

ax.set_axis_off()

plt.title("Polygons identified in the municipality of Bologna - 50 m buffer", fontsize=14)
plt.tight_layout()

# to store
output_path = os.path.join("images", "bologna_polygons_50m.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# plot
plt.show()

## 200 M BUFFER POLYGONS PLOT

In [ ]:
# PREPARATIONS
buffered_200 = []

for el in data_polygons.id.unique():
    
    # here we compute the buffered polygon
    geom_proj = polygons_proj.loc[int(el)].geometry.buffer(200) # add a 100 meters buffer to the polygon
    
    # and look for the data points contained in it
    buffered_polygon = gpd.GeoDataFrame(geometry=[geom_proj], crs="EPSG:32632")
    subset_data_buffered = gpd.sjoin(new_data, buffered_polygon, predicate="within")
    
    buffered_200.append({'id': el, 'geometry': geom_proj, 'POIs': subset_data_buffered.shape[0]})
   

In [ ]:
buffered200_polygons = gpd.GeoDataFrame(buffered_200, crs="EPSG:32632")
buffered200_polygons['area'] = buffered200_polygons.geometry.area /1e6
buffered200_polygons

In [ ]:
# for plot purposes we change the crs system Web Mercator (EPSG:3857) -> to match basemap tiles
buffered100_polygons = buffered100_polygons.to_crs(epsg=3857)
buffered100_polygons.id = buffered100_polygons.id.astype(str)
 
# plot
fig, ax = plt.subplots(figsize=(10, 10))

buffered100_polygons.plot(
    ax=ax,
    column='id',          
    cmap='Set2',        
    alpha=0.5,            
    edgecolor='black',    
    linewidth=0.5,
    legend=True)



ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)  

ax.set_axis_off()

plt.title("Polygons identified in the municipality of Bologna - 100 m buffer", fontsize=14)
plt.tight_layout()

# to store
output_path = os.path.join("images", "bologna_polygons_100m.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# plot
plt.show()

In [ ]:
# PREPARATIONS
buffered_200 = []

for el in data_polygons.id.unique():
    
    # here we compute the buffered polygon
    geom_proj = polygons_proj.loc[int(el)].geometry.buffer(200) # add a 100 meters buffer to the polygon
    
    # and look for the data points contained in it
    buffered_polygon = gpd.GeoDataFrame(geometry=[geom_proj], crs="EPSG:32632")
    subset_data_buffered = gpd.sjoin(new_data, buffered_polygon, predicate="within")
    
    buffered_200.append({'id': el, 'geometry': geom_proj, 'POIs': subset_data_buffered.shape[0]})
   

In [ ]:
buffered200_polygons = gpd.GeoDataFrame(buffered_200, crs="EPSG:32632")
buffered200_polygons['area'] = buffered200_polygons.geometry.area /1e6
buffered200_polygons

In [ ]:
buffered200_polygons = buffered200_polygons.to_crs(epsg=3857)
buffered200_polygons.id = buffered200_polygons.id.astype(str)

## COMPLETE PLOT

In [ ]:


fig, ax = plt.subplots(figsize=(10, 10))

# plot original polygons
data_polygons.plot(
    ax=ax,
    column='id',
    cmap='Set2',
    alpha=0.5,
    edgecolor='black',
    linewidth=0.5,
    legend=True,
)

# plot buffered polygons (transparent fill, red edges for distinction)
buffered100_polygons.boundary.plot(
    ax=ax,
    color='red',
    linewidth=1,
    linestyle='--',
    label='Buffered 100m'
)

# plot buffered polygons (transparent fill, red edges for distinction)
buffered200_polygons.boundary.plot(
    ax=ax,
    color='blue',
    linewidth=1,
    linestyle='--',
    label='Buffered 200m'
)
# add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

ax.set_axis_off()
plt.title("Polygons and Their 100m Buffers - Bologna", fontsize=14)
plt.legend()
plt.tight_layout()

# save if desired
output_path = os.path.join("images", "bologna_polygons_with_buffers.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')

plt.show()
